<a href="https://colab.research.google.com/github/isamdr86/towards-ai/blob/main/notebooks/17-Using_LLMs_to_rank_chunks_as_the_Judge_ir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables


In [1]:
!pip install -q huggingface_hub llama-index==0.10.49 llama-index-vector-stores-chroma==0.1.9 llama-index-llms-gemini==0.1.11 google-generativeai==0.5.4 openai==1.35.3 chromadb==0.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.4 MB/s eta 0

In [2]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [4]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')

# Load a Model


In [5]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(temperature=1, model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Load Indexes


Downloading vector store from Huggingface hub

In [6]:
from huggingface_hub import hf_hub_download
vectorstore = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="vectorstore.zip",repo_type="dataset",local_dir="/content")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vectorstore.zip:   0%|          | 0.00/97.2M [00:00<?, ?B/s]

In [7]:
!unzip /content/vectorstore.zip

Archive:  /content/vectorstore.zip
   creating: ai_tutor_knowledge/
   creating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/length.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/index_metadata.pickle  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/link_lists.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/header.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/data_level0.bin  
  inflating: ai_tutor_knowledge/chroma.sqlite3  


In [8]:
# Load the vector store from the local storage.
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db2 = chromadb.PersistentClient(path="/content/ai_tutor_knowledge")
chroma_collection = db2.get_or_create_collection("ai_tutor_knowledge")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [9]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding

# Create the index based on the vector store.
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [10]:
query_engine = index.as_query_engine(similarity_top_k=10)

res = query_engine.query("Explain how Advance RAG works?")

In [11]:
res.response

'Advance Retrieval-Augmented Generation (RAG) includes various methods that enhance the generative capabilities of large language models (LLMs) by integrating information retrieval processes. The key approaches within Advanced RAG involve distinct strategies such as Self-Reflective RAG (Self-RAG), Corrective RAG (CRAG), and Speculative RAG.\n\n1. **Self-Reflective RAG (Self-RAG)**: This approach utilizes instruction-tuning of LLMs to generate self-reflection tags. These tags guide the models to dynamically retrieve and evaluate the relevance of documents prior to generating responses, improving the contextual quality of the generated content.\n\n2. **Corrective RAG (CRAG)**: This method involves employing an external evaluator to scrutinize and refine the quality of the documents retrieved before the generation phase. This assessment helps ensure that only the most pertinent information is used in the output generation process.\n\n3. **Speculative RAG**: A more recent addition, SPECULA

# RankGPT


In [12]:
from llama_index.core.postprocessor.rankGPT_rerank import RankGPTRerank

rankGPT = RankGPTRerank(top_n=3,llm=Settings.llm, verbose=True)

In [13]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
# The `node_postprocessors` function will be applied to the retrieved nodes.
query_engine = index.as_query_engine(similarity_top_k=10, node_postprocessors=[rankGPT])

res = query_engine.query("Explain how Retrieval Augmented Generation (RAG) works?")

After Reranking, new rank list for nodes: [1, 0, 9, 3, 4, 6, 8, 7, 2, 5]

In [14]:
res.response

'Retrieval-Augmented Generation (RAG) operates through a combination of two primary components: retrieval and generation. \n\n1. **Retrieval Component**:\n   - This component extracts relevant information from external knowledge sources. The process involves two key phases:\n     - **Indexing**: Documents are organized to facilitate efficient retrieval, using techniques such as inverted indexes for sparse retrieval or dense vector encoding for dense retrieval.\n     - **Searching**: Relevant documents are fetched based on user queries, often enhanced by rerankers that improve the relevance ranking of the retrieved documents.\n\n2. **Generation Component**:\n   - The generation phase utilizes the retrieved content alongside the user query to produce coherent and contextually appropriate responses. It includes prompting techniques such as Chain of Thought and Tree of Thought which guide the model to generate better quality responses. \n   - Additionally, during the inferencing phase, the

In [15]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text.strip())
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 1c324686-fad7-4a41-bfd3-d44f9612ca91
Title	 Evaluation of Retrieval-Augmented Generation: A Survey:Research Paper Information
Text	 Authors: Hao Yu, Aoran Gan, Kai Zhang, Shiwei Tong, Qi Liu, and Zhaofeng Liu.Numerous studies of Retrieval-Augmented Generation (RAG) systems have emerged from various perspectives since the advent of Large Language Models (LLMs). The RAG system comprises two primary components: Retrieval and Generation. The retrieval component aims to extract relevant information from various external knowledge sources. It involves two main phases, indexing and searching. Indexing organizes documents to facilitate efficient retrieval, using either inverted indexes for sparse retrieval or dense vector encoding for dense retrieval. The searching component utilizes these indexes to fetch relevant documents based on the user's query, often incorporating the optional rerankers to refine the ranking of the retrieved documents. The generation component utilizes the retr

# Custom Postprocessor


## The `Judger` Function


The following function will query GPT-4o to retrieve the top three nodes that has highest similarity to the asked question.


In [17]:
from pydantic import BaseModel
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts import PromptTemplate


def judger(nodes, query):

    # The model's output template
    class OrderedNodes(BaseModel):
        """A node with the id and assigned score."""

        node_id: list
        score: list

    # Prepare the nodes and wrap them in <NODE></NODE> identifier, as well as the query
    the_nodes = ""
    for idx, item in enumerate(nodes):
        the_nodes += f"<NODE{idx+1}>\nNode ID: {item.node_id}\nText: {item.text}\n</NODE{idx+1}>\n"

    query = "<QUERY>\n{}\n</QUERY>".format(query)

    # Define the prompt template
    prompt_tmpl = PromptTemplate(
        """
    You receive a query along with a list of nodes' text and their ids. Your task is to assign score
    to each node based on its contextually closeness to the given query. The final output is each
    node id along with its proximity score.
    Here is the list of nodes:
    {nodes_list}

    And the following is the query:
    {user_query}

    Score each of the nodes based on their text and their relevancy to the provided query.
    The score must be a decimal number between 0 an 1 so we can rank them."""
    )

    # Define the an instance of GPT-4 and send the request
    llm = OpenAI(model="gpt-4o-mini")
    ordered_nodes = llm.structured_predict(
        OrderedNodes, prompt_tmpl, nodes_list=the_nodes, user_query=query
    )

    return ordered_nodes

## Define Postprocessor


The following class will use the `judger` function to rank the nodes, and filter them based on the ranks.


In [18]:
from typing import List, Optional
from llama_index.core import QueryBundle
from llama_index.core.postprocessor.types import BaseNodePostprocessor
from llama_index.core.schema import NodeWithScore


class OpenaiAsJudgePostprocessor(BaseNodePostprocessor):
    def _postprocess_nodes(
        self, nodes: List[NodeWithScore], query_bundle: Optional[QueryBundle]
    ) -> List[NodeWithScore]:

        r = judger(nodes, query_bundle)

        node_ids = r.node_id
        scores = r.score

        sorted_scores = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
        top_three_nodes = [sorted_scores[i][0] for i in range(3)]

        selected_nodes_id = [node_ids[item] for item in top_three_nodes]

        final_nodes = []
        for item in nodes:
            if item.node_id in selected_nodes_id:
                final_nodes.append(item)

        return final_nodes

In [19]:
judge = OpenaiAsJudgePostprocessor()

## Query Engine with Postprocessor


In [20]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
# The `node_postprocessors` function will be applied to the retrieved nodes.
query_engine = index.as_query_engine(similarity_top_k=5, node_postprocessors=[judge])

res = query_engine.query("Explain how Parameter Efficient Fine tuning (PEFT) works?")

In [21]:
res.response

'Parameter Efficient Fine-Tuning (PEFT) is a technique designed to adapt large pretrained models to various downstream applications without the need to fine-tune all the model parameters, which can be computationally and storage intensive. Instead, PEFT methods focus on fine-tuning only a small number of additional parameters, significantly reducing the computational and storage costs while achieving performance levels comparable to fully fine-tuned models. This approach enhances accessibility for training and storing large language models on consumer hardware.\n\nPEFT integrates seamlessly with popular libraries like Transformers, Diffusers, and Accelerate, streamlining the training and inference processes for large models. By leveraging the expressiveness of pretrained models and adapting only a fraction of their parameters, PEFT provides an efficient alternative to traditional fine-tuning methods.'

In [22]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 6be88fa3-2f8b-43e7-aba0-d874b39809fc
Title	 FourierFT: Discrete Fourier Transformation Fine-Tuning
Text	 # FourierFT: Discrete Fourier Transformation Fine-Tuning[FourierFT](https://huggingface.co/papers/2405.03003) is a parameter-efficient fine-tuning technique that leverages Discrete Fourier Transform to compress the model's tunable weights. This method outperforms LoRA in the GLUE benchmark and common ViT classification tasks using much less parameters.FourierFT currently has the following constraints:- Only `nn.Linear` layers are supported.- Quantized layers are not supported.If these constraints don't work for your use case, consider other methods instead.The abstract from the paper is:> Low-rank adaptation (LoRA) has recently gained much interest in fine-tuning foundation models. It effectively reduces the number of trainable parameters by incorporating low-rank matrices A and B to represent the weight change, i.e., Delta W=BA. Despite LoRA's progress, it faces storage ch